# Process results we got by confronting generators and discriminators

## Preprocessing

In [1]:
%cd ../..

/Users/antoinestutz/Documents/Cours/Deepl/Projet/cartoongan


In [35]:
import json
import numpy as np

from scripts import config

In [4]:
with open(config.ADVERSARIAL_SCORES_PATH, "r", encoding="utf-8") as f:
    results_graph = json.load(f)

In [36]:
# First remove runs if they have a None score
issues_with = set()

for run_id1 in results_graph:
    for run_id2 in results_graph[run_id1]:
        if run_id2 not in results_graph or results_graph[run_id1][run_id2] is None or np.isnan(results_graph[run_id1][run_id2]):
            issues_with.add(run_id2)

issues_with = list(issues_with)
issues_with

['2022_03_28-18_15_16_40',
 '2022_03_28-18_15_16_72',
 '2022_03_28-18_15_16_50',
 '2022_03_28-18_15_16_20',
 '2022_03_28-18_15_16_30',
 '2022_03_28-18_15_16_60']

In [37]:
# Then only keep the models processed by all
intersection = set(results_graph[list(results_graph.keys())[0]].keys())

for run_id1 in results_graph:
    results_graph[run_id1] = {k: v for k, v in results_graph[run_id1].items() if k not in issues_with}
    intersection = intersection & set(results_graph[run_id1].keys())

print(len(intersection))
results_graph = {k: {k1: v1 for k1, v1 in v.items() if k1 in intersection} for k, v in results_graph.items() if k in intersection}

187


In [90]:
scores_matrix = np.zeros((len(results_graph), len(results_graph)))
int_to_name = list(results_graph.keys())
name_to_int = {k: v for v, k in enumerate(int_to_name)}

for run_id1 in results_graph:
    for run_id2 in results_graph[run_id1]:
        scores_matrix[name_to_int[run_id1], name_to_int[run_id2]] = results_graph[run_id1][run_id2]

In [114]:
to_remove = ["2022_03_31-18_37_59_gpK","2022_04_04-18_27_57_veL","2022_04_04-03_19_26_QAj","2022_03_31-18_38_39_AqQ","2022_03_31-18_56_57_xfY","2022_04_01-04_25_45_DEW","2022_04_01-18_56_08_GUY"]

indexes_to_remove = []
for run_id in int_to_name:
    for run_id_to_del in to_remove:
        if run_id_to_del in run_id:
            indexes_to_remove.append(name_to_int[run_id])

int_to_name_filtered = [run_id for i, run_id in enumerate(int_to_name) if i not in indexes_to_remove]
name_to_int_filtered = {k: v for v, k in enumerate(int_to_name_filtered)}

scores_matrix_filtered = np.delete(scores_matrix, indexes_to_remove, axis=0)
scores_matrix_filtered = np.delete(scores_matrix_filtered, indexes_to_remove, axis=1)

## First results: best sum of scores

In [103]:
gen_results_simple = scores_matrix.sum(axis=1)
best_generators_simple = [(run_id, gen_results_simple[name_to_int[run_id]]) for run_id in int_to_name]
best_generators_simple = sorted(best_generators_simple, key=lambda x: x[1], reverse=True)
best_generators_simple

[('2022_03_31-18_37_59_gpK_20', 48.12014228335759),
 ('2022_03_31-18_37_59_gpK_10', 47.986227595971584),
 ('2022_03_31-18_37_59_gpK_114', 47.9478303422998),
 ('2022_03_31-18_37_59_gpK_50', 47.9396310155415),
 ('2022_03_31-18_37_59_gpK_40', 47.91638302972961),
 ('2022_03_31-18_37_59_gpK_60', 47.89434835990753),
 ('2022_04_04-18_27_57_veL_50', 47.884427450593165),
 ('2022_03_31-18_37_59_gpK_30', 47.854578526359795),
 ('2022_03_31-18_37_59_gpK_5', 47.817656172243076),
 ('2022_03_31-18_38_39_AqQ_50', 47.777338873265805),
 ('2022_04_04-18_27_57_veL_60', 47.75159571669687),
 ('2022_04_04-18_27_57_veL_40', 47.75063927210057),
 ('2022_03_28-02_02_43_97', 47.75020048119801),
 ('2022_04_04-18_27_57_veL_10', 47.748473136730325),
 ('2022_03_28-02_02_43_60', 47.742494207403254),
 ('2022_04_04-03_19_26_QAj_10', 47.74225346003852),
 ('2022_03_29-04_12_27_10', 47.74189731196516),
 ('2022_04_03-20_03_58_Sko_20', 47.73861710675864),
 ('2022_03_28-18_15_38_30', 47.736973004103255),
 ('2022_03_31-18_56_57

In [105]:
disc_results_simple = (1-scores_matrix).sum(axis=0)
best_discriminators_simple = [(run_id, disc_results_simple[name_to_int[run_id]]) for run_id in int_to_name]
best_discriminators_simple = sorted(best_discriminators_simple, key=lambda x: x[1], reverse=True)
best_discriminators_simple

[('2022_04_01-18_56_08_GUY_50', 186.9998228757903),
 ('2022_04_01-18_56_08_GUY_60', 186.99982253654235),
 ('2022_04_01-18_56_08_GUY_111', 186.99970251328625),
 ('2022_04_01-18_56_08_GUY_40', 186.99939034568493),
 ('2022_04_01-18_56_08_GUY_20', 186.9921883820137),
 ('2022_04_01-18_56_08_GUY_5', 186.980783442743),
 ('2022_04_01-04_25_45_DEW_50', 186.97942699912957),
 ('2022_04_01-18_56_08_GUY_30', 186.95539493997487),
 ('2022_04_01-04_25_45_DEW_60', 186.94998763762993),
 ('2022_04_01-18_56_08_GUY_10', 186.93900817825912),
 ('2022_04_01-04_25_45_DEW_20', 186.91277921402508),
 ('2022_04_01-04_25_45_DEW_40', 186.82957019954037),
 ('2022_04_01-04_25_45_DEW_30', 186.35895050374094),
 ('2022_04_01-04_25_45_DEW_10', 186.21997884926378),
 ('2022_04_01-04_25_45_DEW_5', 186.0263084371341),
 ('2022_03_29-21_05_31_60', 185.89390752341137),
 ('2022_04_01-04_25_45_DEW_95', 185.71993539954997),
 ('2022_03_28-02_02_38_20', 185.644642504491),
 ('2022_03_29-21_05_24_10', 185.64049948905603),
 ('2022_03_29

In [109]:
gen_results_simple_filtered = scores_matrix_filtered.sum(axis=1)
best_generators_simple_filtered = [(run_id, gen_results_simple_filtered[name_to_int_filtered[run_id]]) for run_id in int_to_name_filtered]
best_generators_simple_filtered = sorted(best_generators_simple_filtered, key=lambda x: x[1], reverse=True)
best_generators_simple_filtered

[('2022_03_28-02_02_43_97', 33.2978575971676),
 ('2022_03_28-02_02_43_60', 33.292998783965615),
 ('2022_03_29-04_12_27_10', 33.29213865697384),
 ('2022_04_03-20_03_58_Sko_20', 33.28918814724312),
 ('2022_03_28-18_15_38_30', 33.287979234405796),
 ('2022_03_28-02_02_43_50', 33.28085186504758),
 ('2022_03_29-04_12_27_60', 33.27940505335575),
 ('2022_03_29-21_05_31_40', 33.27665927542838),
 ('2022_03_28-02_01_58_50', 33.27589396818075),
 ('2022_03_28-02_02_28_30', 33.27535497986246),
 ('2022_03_28-02_02_43_10', 33.27488909896928),
 ('2022_03_27-20_51_29_30', 33.27460083195474),
 ('2022_03_29-21_05_31_10', 33.273423116607596),
 ('2022_04_04-02_53_39_YpC_5', 33.27158110255841),
 ('2022_03_27-20_52_24_19', 33.270377729367475),
 ('2022_03_27-20_51_29_5', 33.26991779576056),
 ('2022_04_03-20_04_09_Bkv_20', 33.269387342361746),
 ('2022_03_29-21_05_31_74', 33.26927977649029),
 ('2022_03_28-18_15_38_5', 33.26823808324989),
 ('2022_03_27-20_51_13_30', 33.267392247216776),
 ('2022_03_28-02_01_58_20'

In [117]:
disc_results_simple_filtered = (1-scores_matrix_filtered).sum(axis=0)
best_discriminators_simple_filtered = [(run_id, disc_results_simple_filtered[name_to_int_filtered[run_id]]) for run_id in int_to_name_filtered]
best_discriminators_simple_filtered = sorted(best_discriminators_simple_filtered, key=lambda x: x[1], reverse=True)
best_discriminators_simple_filtered

[('2022_03_29-21_05_31_60', 132.21450746962802),
 ('2022_03_28-02_02_38_20', 132.03664621184578),
 ('2022_03_29-21_05_24_10', 132.03453287160957),
 ('2022_03_29-04_14_51_10', 131.9307372572832),
 ('2022_03_27-20_51_29_10', 131.85177273163572),
 ('2022_03_29-21_05_24_38', 131.73457734561526),
 ('2022_04_04-02_53_39_YpC_5', 131.52310851947405),
 ('2022_03_29-04_12_27_60', 131.48027649186554),
 ('2022_03_28-18_15_38_10', 131.19159527588633),
 ('2022_03_30-18_24_30_GnA_20', 131.11024673492648),
 ('2022_03_30-18_24_30_GnA_10', 130.56311198696494),
 ('2022_03_28-18_23_16_50', 130.4629662045278),
 ('2022_03_28-02_02_38_60', 130.45797896850854),
 ('2022_03_29-21_05_24_20', 130.38148354124277),
 ('2022_03_28-18_15_38_40', 130.36533820247274),
 ('2022_03_27-20_51_13_5', 130.35934531614186),
 ('2022_03_27-20_51_43_20', 130.1987585157156),
 ('2022_03_27-20_51_29_20', 130.06214410262183),
 ('2022_03_27-20_51_13_10', 129.79103134665638),
 ('2022_03_28-02_02_28_20', 129.78333725482224),
 ('2022_03_27

## Second results: HITS

In [126]:
def generate_hits_scores(scores_matrix, epsilon=1e-6):
    gen_matrix = scores_matrix
    disc_matrix = 1-scores_matrix

    gen = np.ones(scores_matrix.shape[0])
    disc = np.ones(scores_matrix.shape[1])

    eps = epsilon + 1
    while eps > epsilon:
        new_gen = np.dot(gen_matrix, disc)
        new_disc = np.dot(gen, disc_matrix)
        new_gen = new_gen/np.linalg.norm(new_gen)
        new_disc = new_disc/np.linalg.norm(new_disc)
        eps = max(np.linalg.norm(new_gen - gen), np.linalg.norm(new_disc - disc))
        gen = new_gen
        disc = new_disc
    return gen, disc

In [127]:
gen, disc = generate_hits_scores(scores_matrix)
best_generators = [(run_id, gen[name_to_int[run_id]]) for run_id in int_to_name]
best_generators = sorted(best_generators, key=lambda x: x[1], reverse=True)
best_generators

[('2022_03_31-18_37_59_gpK_20', 0.07391603544783316),
 ('2022_03_31-18_37_59_gpK_10', 0.07366436164705799),
 ('2022_03_31-18_37_59_gpK_114', 0.07359805536965029),
 ('2022_03_31-18_37_59_gpK_50', 0.07357157757596632),
 ('2022_03_31-18_37_59_gpK_40', 0.07352791508959355),
 ('2022_03_31-18_37_59_gpK_60', 0.07348563096011101),
 ('2022_04_04-18_27_57_veL_50', 0.07347193230336971),
 ('2022_03_31-18_37_59_gpK_30', 0.07341396393052699),
 ('2022_03_31-18_37_59_gpK_5', 0.07334519783915201),
 ('2022_03_31-18_38_39_AqQ_50', 0.07327022372417433),
 ('2022_03_28-02_02_43_97', 0.07322863497900428),
 ('2022_04_04-18_27_57_veL_60', 0.07322212831718394),
 ('2022_04_04-18_27_57_veL_40', 0.07321897830377744),
 ('2022_04_04-18_27_57_veL_10', 0.07321666730241004),
 ('2022_03_28-02_02_43_60', 0.07320700525813907),
 ('2022_04_04-03_19_26_QAj_10', 0.07320536824723883),
 ('2022_03_29-04_12_27_10', 0.07320511927691628),
 ('2022_04_03-20_03_58_Sko_20', 0.07319703134060726),
 ('2022_03_28-18_15_38_30', 0.0731959186

In [128]:
best_discriminators = [(run_id, disc[name_to_int[run_id]]) for run_id in int_to_name]
best_discriminators = sorted(best_discriminators, key=lambda x: x[1], reverse=True)
best_discriminators

[('2022_04_01-18_56_08_GUY_50', 0.09292596812854198),
 ('2022_04_01-18_56_08_GUY_60', 0.09292596795882975),
 ('2022_04_01-18_56_08_GUY_111', 0.09292590831575784),
 ('2022_04_01-18_56_08_GUY_40', 0.0929257531915795),
 ('2022_04_01-18_56_08_GUY_20', 0.09292217427987758),
 ('2022_04_01-18_56_08_GUY_5', 0.09291650676947526),
 ('2022_04_01-04_25_45_DEW_50', 0.0929158327877459),
 ('2022_04_01-18_56_08_GUY_30', 0.0929038904589736),
 ('2022_04_01-04_25_45_DEW_60', 0.09290120319643286),
 ('2022_04_01-18_56_08_GUY_10', 0.09289574740778458),
 ('2022_04_01-04_25_45_DEW_20', 0.09288271308515561),
 ('2022_04_01-04_25_45_DEW_40', 0.09284136393148477),
 ('2022_04_01-04_25_45_DEW_30', 0.09260749870797538),
 ('2022_04_01-04_25_45_DEW_10', 0.09253843971701407),
 ('2022_04_01-04_25_45_DEW_5', 0.09244219916736851),
 ('2022_03_29-21_05_31_60', 0.09237639826636385),
 ('2022_04_01-04_25_45_DEW_95', 0.09228993762658998),
 ('2022_03_28-02_02_38_20', 0.09225253374128647),
 ('2022_03_29-21_05_24_10', 0.0922504713

In [129]:
gen_filtered, disc_filtered = generate_hits_scores(scores_matrix_filtered)
best_generators_filtered = [(run_id, gen_filtered[name_to_int_filtered[run_id]]) for run_id in int_to_name_filtered]
best_generators_filtered = sorted(best_generators_filtered, key=lambda x: x[1], reverse=True)
best_generators_filtered

[('2022_03_28-02_02_43_97', 0.0868990840086168),
 ('2022_03_28-02_02_43_60', 0.0868717370115068),
 ('2022_03_29-04_12_27_10', 0.08686762415701826),
 ('2022_04_03-20_03_58_Sko_20', 0.08685503907837179),
 ('2022_03_28-18_15_38_30', 0.08685293513435384),
 ('2022_03_28-02_02_43_50', 0.08683046941902335),
 ('2022_03_29-04_12_27_60', 0.08682431371436423),
 ('2022_03_29-21_05_31_40', 0.08681573733682875),
 ('2022_03_28-02_02_43_10', 0.08681491510316035),
 ('2022_03_28-02_01_58_50', 0.08681433785172132),
 ('2022_03_28-02_02_28_30', 0.08681176146572576),
 ('2022_03_27-20_51_29_30', 0.0868060860933022),
 ('2022_03_29-21_05_31_10', 0.0867990543099989),
 ('2022_04_04-02_53_39_YpC_5', 0.08679776577626774),
 ('2022_03_27-20_52_24_19', 0.08679610288034134),
 ('2022_03_27-20_51_29_5', 0.08679542737011932),
 ('2022_03_29-21_05_31_74', 0.08679147151903209),
 ('2022_04_03-20_04_09_Bkv_20', 0.08678802311134338),
 ('2022_03_28-18_15_38_5', 0.08678793814539904),
 ('2022_03_28-02_01_58_20', 0.086785796338601

In [130]:
best_discriminators_filtered = [(run_id, disc_filtered[name_to_int_filtered[run_id]]) for run_id in int_to_name_filtered]
best_discriminators_filtered = sorted(best_discriminators_filtered, key=lambda x: x[1], reverse=True)
best_discriminators_filtered

[('2022_03_29-21_05_31_60', 0.10835488436751749),
 ('2022_03_28-02_02_38_20', 0.10820912071463842),
 ('2022_03_29-21_05_24_10', 0.10820738833656367),
 ('2022_03_29-04_14_51_10', 0.10812232269665921),
 ('2022_03_27-20_51_29_10', 0.10805761023599116),
 ('2022_03_29-21_05_24_38', 0.10796156421732192),
 ('2022_04_04-02_53_39_YpC_5', 0.10778825725480025),
 ('2022_03_29-04_12_27_60', 0.10775315254456695),
 ('2022_03_28-18_15_38_10', 0.10751656849460144),
 ('2022_03_30-18_24_30_GnA_20', 0.10744990102153167),
 ('2022_03_30-18_24_30_GnA_10', 0.10700150356054451),
 ('2022_03_28-18_23_16_50', 0.1069194285174099),
 ('2022_03_28-02_02_38_60', 0.10691534354909236),
 ('2022_03_29-21_05_24_20', 0.10685264880686261),
 ('2022_03_28-18_15_38_40', 0.10683942291643958),
 ('2022_03_27-20_51_13_5', 0.10683450803023252),
 ('2022_03_27-20_51_43_20', 0.10670290077098545),
 ('2022_03_27-20_51_29_20', 0.10659093861504461),
 ('2022_03_27-20_51_13_10', 0.10636875406231874),
 ('2022_03_28-02_02_28_20', 0.10636244498